# IMPORT LIBRARIES

In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

# KEYPOINTS USING MP HOLISTIC

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawings = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):  
    #Draw left hand connections
    mp_drawings.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                               mp_drawings.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                               mp_drawings.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    #Draw right hand connections 
    mp_drawings.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                               mp_drawings.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),  
                               mp_drawings.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))  

In [13]:
cap = cv2.VideoCapture(0)
#Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # MAke Decision
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_landmarks(image, results)

        # Show to Screen
        cv2.imshow('OpenCV Feed', image)

        # Break the code
        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [15]:
len(results.left_hand_landmarks.landmark)

21

# EXTRACT KEY POINTS

In [20]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh,rh])

In [22]:
extract_keypoints(results).shape

(126,)

# SETUP FOLDER FOR DATA COLLECTION

In [23]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('CC_DATA')

# ACTIONS that we try to detect
actions = np.array(['click','scroll','swipe','pause','tabchange'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 60

In [24]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

# COLLECT KEYPOINTS FOR TRAINING AND TESTING

In [25]:
cap = cv2.VideoCapture(0)
#Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # MAke Decision
                image, results = mediapipe_detection(frame, holistic)
                print(results)

                # Draw landmarks
                draw_landmarks(image, results)

                # Apply wait Logic
                if frame_num == 0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
                    cv2.putText(image,f'COLLECTING FRAMES FOR {action} Video Number {sequence}',(15,12),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.waitKey(2500)
                else:
                    cv2.putText(image,f'COLLECTING FRAMES FOR {action} Video Number {sequence}',(15,12),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path, keypoints)

                # Show to Screen
                cv2.imshow('OpenCV Feed', image)

                # Break the code
                if cv2.waitKey(10) & 0xff == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# PREPROCESS DATA AND CREATE FEATURE AND LABELS

In [26]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [27]:
label_map = {'click':0, 'scroll':1, 'swipe':2, 'pause':3, 'tabchange':4}

In [28]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),f'{frame_num}.npy'))
            window.append(res)

        sequences.append(window)
        labels.append(label_map[action])

In [29]:
X = np.array(sequences)

In [30]:
X.shape

(150, 60, 126)

In [31]:
y = to_categorical(labels).astype(int)

In [39]:
y.shape

(150, 5)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05)

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard

In [45]:
log_dir = os.path.join('CC_Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [46]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation="tanh",input_shape=(60,126)))
model.add(LSTM(128,return_sequences=True,activation="tanh"))
model.add(LSTM(64,return_sequences=False,activation="tanh"))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(32,activation="relu"))
model.add(Dense(actions.shape[0],activation="softmax"))

In [47]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [48]:
model.fit(X_train,y_train,epochs=300,batch_size=32,shuffle=True,callbacks=[tb_callback])

Epoch 1/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - categorical_accuracy: 0.2254 - loss: 1.6183
Epoch 2/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - categorical_accuracy: 0.2113 - loss: 1.6152
Epoch 3/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - categorical_accuracy: 0.3451 - loss: 1.5983
Epoch 4/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - categorical_accuracy: 0.3310 - loss: 1.5850
Epoch 5/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - categorical_accuracy: 0.3310 - loss: 1.5303
Epoch 6/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - categorical_accuracy: 0.3239 - loss: 1.4865
Epoch 7/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - categorical_accuracy: 0.4225 - loss: 1.3798
Epoch 8/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - categorical_accuracy: 0.4859 - loss: 1.2965
Epoch 9/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - categorical_accuracy: 0.5000 - loss: 1.2118
Epoch 10/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - categorical_accuracy: 0.5775 - loss: 1.0762
Epoch 11/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0

KeyboardInterrupt: 

In [73]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step


# MODEL SAVING

In [74]:
model.save('CC_action.h5')

In [75]:
model.load_weights('CC_action.h5')

# EVALUATION USING CONFUSION MATRIX AND ACCURACY

In [76]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [77]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [78]:
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [79]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[4, 1],
        [0, 3]],

       [[7, 0],
        [1, 0]],

       [[7, 0],
        [0, 1]],

       [[5, 0],
        [0, 3]]], dtype=int64)

In [80]:
accuracy_score(ytrue,yhat)

0.875

# TEST IN REAL TIME

In [81]:
from scipy import stats

In [99]:
colors = [
    (245, 117, 16),   # Orange
    (117, 245, 16),   # Green
    (16, 117, 245),   # Blue
    (255, 0, 255),    # Magenta
    (0, 255, 255)     # Cyan
]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [100]:
sequence = []
sentence = []
threshold = 0.7


cap = cv2.VideoCapture(0)
#Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # MAke Decision
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_landmarks(image, results)

        # Prediction Logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])

        # viz logic
        if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                        sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5:
            sentence = sentence[-5:]

        # viz probabilities
        image = prob_viz(res,actions,image,colors)
        
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image, ' '.join(sentence),(3,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2,cv2.LINE_AA)

        # Show to Screen
        cv2.imshow('OpenCV Feed', image)

        # Break the code
        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [101]:
!pip install pyautogui

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (

  DEPRECATION: Building 'pygetwindow' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pygetwindow'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'pytweening' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pytweening'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'mouseinfo' using the legacy setup.py bdist_wh

In [107]:
import pyautogui
import webbrowser
import time
import random
import math

def perform_action(prediction):
    if prediction == 'click':
        pyautogui.click()
        print("Clicked")

    elif prediction == 'scroll':
        pyautogui.scroll(-500)  # scroll down; use +500 to scroll up
        print("Scrolled")

    elif prediction == 'swipe':
        # Swipe cursor around the screen in a smooth circular motion
        x, y = pyautogui.position()
        for angle in range(0, 360, 30):
            radius = 100
            new_x = x + int(radius * math.cos(math.radians(angle)))
            new_y = y + int(radius * math.sin(math.radians(angle)))
            pyautogui.moveTo(new_x, new_y, duration=0.05)
        print("Swiped")

    elif prediction == 'pause':
        pyautogui.press('space')  # commonly pauses/resumes videos
        print("Paused")

    elif prediction == 'tabchange':
        pyautogui.hotkey('ctrl', 'tab')  # switch to next tab
        print("Tab Changed")

    else:
        print("Unknown Action")


In [108]:
last_prediction = ''
last_time = 0
cooldown = 2  # seconds

def should_trigger(pred, conf, threshold=0.8):
    global last_prediction, last_time
    if conf > threshold and (pred != last_prediction or time.time() - last_time > cooldown):
        last_prediction = pred
        last_time = time.time()
        return True
    return False


In [109]:
sequence = []
sentence = []
threshold = 0.7


cap = cv2.VideoCapture(0)
#Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # MAke Decision
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_landmarks(image, results)

        # Prediction Logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        # Trigger Action
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])

            conf = np.max(res)
            pred_class = actions[np.argmax(res)]

            if should_trigger(pred_class, conf):
                perform_action(pred_class)

        # viz logic
        if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                        sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5:
            sentence = sentence[-5:]

        # viz probabilities
        image = prob_viz(res,actions,image,colors)
        
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image, ' '.join(sentence),(3,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2,cv2.LINE_AA)

        # Show to Screen
        cv2.imshow('OpenCV Feed', image)

        # Break the code
        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [106]:
import pyautogui
import time

print("Move your cursor to the desired location in 5 seconds...")
time.sleep(5)
print("Your cursor is at:", pyautogui.position())


Move your cursor to the desired location in 5 seconds...
Your cursor is at: Point(x=1175, y=348)
